In [1]:
from bs4 import BeautifulSoup 
import re
import urllib 
import requests
import string
import pandas as pd

In [34]:
url = 'http://seinfeld.wikia.com/wiki/List_of_Seinfeld_sayings'

source_code = requests.get(url)
html = source_code.text
#print(html)

catch_phrases = re.findall(r'<b>(.*?)</b>', html)

clean_phrases = []
for phrase in catch_phrases:
    phrase = phrase.lower()
    clean_phrases.append(re.sub(r'<.*?>|\"', '', phrase))
print(clean_phrases)

['1st and 1st', 'anti-dentite', 'babka', 'baby', 'bad breaker-upper', 'baldist', 'boys', 'bottle wipe', 'cantstandya!', 'cartwright!', 'close talker', 'coffee table book about coffee tables', 'conjugal visit sex', 'costanza leave behind', 'coup de toe', 'delicate genius', 'double-dipper', 'dry heave set to music', 'even steven', 'festivus', 'fugitive sex', '“get out!”', 'getting upset', 'giddyup!', 'go!', 'gore-tex', 'hand sandwich', 'happy, pappy?', 'hello!', 'hello, newman', 'hello, vargas', 'high talker', 'hipster doofus', 'hoochie mama!', 'the human fund: money for people', 'i am aware', 'i was in the pool!', 'in the vault', "it's not you, it's me", 'john-john', 'kavorka', 'kibosh ', 'laughing and lying', 'low talker', 'lupus?! is it lupus?!', 'make-up sex', 'manssiere/bro', 'man hands', 'master of my domain', 'maybe the dingo ate your baby!', 'mimbo', 'moops', 'mulva', 'newman!', 'no soup for you!', "not that there's anything wrong with that", 'oh, moses, smell the roses!', 'oh, t

In [11]:
print(clean_phrases[-7])

Yada yada yada


In [12]:
dialogue = pd.read_csv('C:\\Users\\Nate\\Documents\\GitHub\\CUNY_DATA_620_GROUP\\Final_Project\\Data\\Seinfield_DialogueSUBSET.csv')

In [40]:
print(dialogue.head())
print(dialogue.loc[:2,'Dialogue'])

  Character                                           Dialogue  DialogueIndex  \
0     JERRY  Do you know what this is all about? Do you kno...            1.0   
1     JERRY  (pointing at Georges shirt) See, to me, that b...            1.0   
2    GEORGE                                   Are you through?            2.0   
3     JERRY             You do of course try on, when you buy?            3.0   
4    GEORGE  Yes, it was purple, I liked it, I dont actuall...            4.0   

     SEID  SceneNum  
0  S01E00  S01E00_1  
1  S01E00  S01E00_2  
2  S01E00  S01E00_2  
3  S01E00  S01E00_2  
4  S01E00  S01E00_2  
0    Do you know what this is all about? Do you kno...
1    (pointing at Georges shirt) See, to me, that b...
2                                     Are you through?
Name: Dialogue, dtype: object


In [41]:
for phrase in clean_phrases:
    utterances = 0 
    for row in dialogue.loc[:,'Dialogue']:
        row = str(row).lower() # not all rows are of type string
        utterances = utterances + len(re.findall(phrase, row))
        #if utterances > 0:
    print("Matches found, " + str(phrase) + ': ' + str(utterances))

Matches found, 1st and 1st: 0
Matches found, anti-dentite: 4
Matches found, babka: 20
Matches found, baby: 195
Matches found, bad breaker-upper: 0
Matches found, baldist: 1
Matches found, boys: 65
Matches found, bottle wipe: 0
Matches found, cantstandya!: 0
Matches found, cartwright!: 4
Matches found, close talker: 1
Matches found, coffee table book about coffee tables: 4
Matches found, conjugal visit sex: 2
Matches found, costanza leave behind: 0
Matches found, coup de toe: 0
Matches found, delicate genius: 5
Matches found, double-dipper: 0
Matches found, dry heave set to music: 1
Matches found, even steven: 2
Matches found, festivus: 28
Matches found, fugitive sex: 2
Matches found, “get out!”: 0
Matches found, getting upset: 4
Matches found, giddyup!: 0
Matches found, go!: 44
Matches found, gore-tex: 6
Matches found, hand sandwich: 1
Matches found, happy, pappy?: 3
Matches found, hello!: 30
Matches found, hello, newman: 8
Matches found, hello, vargas: 0
Matches found, high talker: 4
